In [4]:
!pip install pandas openpyxl
!pip install bitsandbytes==0.38.1
!pip install torch transformers==4.31.0 peft==0.4.0 sentencepiece bitsandbytes accelerate
!pip install huggingface-hub==0.27.0

In [1]:
import torch
import pandas as pd
import json

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from safetensors.torch import save_file
import torch

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    load_in_8bit_fp32_cpu_offload=True  
)

model_name_or_path = "lianghsun/Llama-3.2-Taiwan-3B-Instruct"

token = "your_key"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_auth_token=token)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    use_auth_token=token,
    device_map="auto",  
    quantization_config=quantization_config  
)


/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.23it/s]


In [ ]:
import pandas as pd
import json

ai_path = './AI.xlsx'
ai1000_path = './AI1000.xlsx'

ai_output_path = './AI.json'
ai1000_output_path = './AI1000.json'

def ai_convert_to_json(input_path, output_path):
    df = pd.read_excel(input_path)
    
    data = []
    for _, row in df.iterrows():
        entry = {
            "文章": row['文章'],
            "問題": row['問題'],
            "選項1": row['選項1'],
            "選項2": row['選項2'],
            "選項3": row['選項3'],
            "選項4": row['選項4'],
            "正確答案": row['正確答案'] 
        }
        data.append(entry)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"JSON 文件: {output_path}")

def qa_convert_to_json(input_path, output_path):
    df = pd.read_excel(input_path)
    
    data = []
    for _, row in df.iterrows():
        entry = {
            "題號": row["題號"],
            "文章": row['文章'],
            "問題": row['問題'],
            "選項1": row['選項1'],
            "選項2": row['選項2'],
            "選項3": row['選項3'],
            "選項4": row['選項4'],
        }
        data.append(entry)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

    print(f"JSON 文件: {output_path}")

ai_convert_to_json(ai_path, ai_output_path)

qa_convert_to_json(ai1000_path, ai1000_output_path)


JSON 文件: ./AI.json
JSON 文件: ./AI1000.json


In [2]:
def generate_prompt(text):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text}

### Response:"""

text = """
        文章: 【台江國家公園為台灣第8座國家公園，境內有廣大河口、潟湖、魚塭及紅樹林等濕地環境，也有古蹟、廟宇、民俗等文史資源及漁鹽產業變遷的文化地景。今年春節期間，台江國家公園管理處於大年初一至初五，提供「台江國家公園與綠建築」免費導覽解說，還有黑面琵鷺免費的生態解說，邀請大家春節期間闔家一起遊台江。】\n"
        問題: 台江國家公園為台灣第幾座國家公園?\n
        選項如下：
        選項1: 第4座\n
        選項2: 第6座\n
        選項3: 第8座\n
        選項4: 第5座\n
        根據以上「文章」中提到的內容，「問題」正確答案應該1~4中第幾個「選項」，請嚴格遵守僅回答1,2,3,4中的其中一個數字，不要多做說明?
       """
prompt = generate_prompt(text)
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    output_ids = base_model.generate(
        input_ids=input_ids,
        max_new_tokens=128,
        temperature=0.5,
        top_k=20,
        top_p=0.9,
        repetition_penalty=1.15
    ).cuda()

output = tokenizer.decode(output_ids[0], skip_special_tokens=True, return_full_text=False)
result = output.split("### Response:")[-1].strip()
print(result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


選項3: 第8座

根據文章內容，Taipei Jiang National Park被提到是台灣第八座國家公園，因此選項3是正確答案。請注意，問題要求從給定的選項中選出一個最接近答案的數字，而不是直接引用文章中的具體內容。因此，選項3是唯一符合條件的選項。 lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer
import json
import pandas as pd
from sklearn.model_selection import train_test_split

ai_output_path = './AI.json'
ai1000_output_path = './AI1000.json'

with open(ai_output_path, 'r', encoding='utf-8') as f:
    ai = json.load(f)

for item in ai:
    for key, value in item.items():
        if not isinstance(value, str): 
            item[key] = str(value)  

train_df, valid_df = train_test_split(ai, test_size=0.1, random_state=42)

train_df = Dataset.from_list(train_df)
valid_df = Dataset.from_list(valid_df)

def generate_text_fields(example):
    example["input_text"] = (
        f"文章: 【{example['文章']}】\n"
        f"問題: {example['問題']}\n"
        f"選項如下：\n選項1：{example['選項1']}\n選項2：{example['選項2']}\n選項3：{example['選項3']}\n選項4：{example['選項4']}\n"
        "根據以上「文章」中提到的內容，「問題」正確答案應該1~4中第幾個「選項」，請嚴格遵守僅回答1,2,3,4中的其中一個數字，不要多做說明?"
    )
    example["target_text"] = str(example["正確答案"])
    return example

tr_processed_dataset = train_df.map(generate_text_fields)
vd_processed_dataset = valid_df.map(generate_text_fields)

Map: 100%|██████████| 1355/1355 [00:00<00:00, 31314.57 examples/s]


In [4]:
tr_processed_dataset[1]

{'文章': '年農曆春節有6天假期，不少民眾規劃出國度假或返鄉過節，臺北市動物保護處提醒要幫家中寵物安排寵物旅館的民眾，先上網查詢合格業者，寄宿前為寵物施打晶片和疫苗，並簽訂契約，讓寵物也能快樂過好年。\n\n記者劉碧薇報導------\n\n臺北市動保處表示，選擇寵物旅館首先要上網查詢依「特定寵物業管理辦法」領有營業許可證的合法犬類寄宿事業，不過該法目前適用的寵物種類只有犬隻，貓、兔等，其他寵物還沒有相關規範。另外動保處也呼籲，過年期間犬貓頻繁進出寵物旅館及寵物美容店，是寵物走失及疾病傳染等高峰期，因此寄宿前務必先辦理寵物登記及施打疫苗，並簽定契約。北市動保處產業保育組長許萌芳說------\n動保處表示，往年春節後寵物相關消費爭議暴增，因此飼主在送寵物去寄宿前應先充分與業者溝通，詢問過年期間寵物旅館業者排班方式及時間，並簽訂契約，以書面註明寄養方式，另外應向業者主動表明寵物的個性、相關病史、進食與用藥習慣，以保障雙方權利義務，並須事先約定寵物意外、就醫，與醫療費用分攤方式，可避免後續衍生的消費糾紛。\n\n\n',
 '問題': '何時為寵物走失及疾病傳染等高峰期？',
 '選項1': '過年',
 '選項2': '端午',
 '選項3': '中秋',
 '選項4': '清明',
 '正確答案': '1',
 'input_text': '文章: 【年農曆春節有6天假期，不少民眾規劃出國度假或返鄉過節，臺北市動物保護處提醒要幫家中寵物安排寵物旅館的民眾，先上網查詢合格業者，寄宿前為寵物施打晶片和疫苗，並簽訂契約，讓寵物也能快樂過好年。\n\n記者劉碧薇報導------\n\n臺北市動保處表示，選擇寵物旅館首先要上網查詢依「特定寵物業管理辦法」領有營業許可證的合法犬類寄宿事業，不過該法目前適用的寵物種類只有犬隻，貓、兔等，其他寵物還沒有相關規範。另外動保處也呼籲，過年期間犬貓頻繁進出寵物旅館及寵物美容店，是寵物走失及疾病傳染等高峰期，因此寄宿前務必先辦理寵物登記及施打疫苗，並簽定契約。北市動保處產業保育組長許萌芳說------\n動保處表示，往年春節後寵物相關消費爭議暴增，因此飼主在送寵物去寄宿前應先充分與業者溝通，詢問過年期間寵物旅館業者排班方式及時間，並簽訂契約，以書面註明寄養方式，另外應向業者主動表明寵物的個性、相關病史、進食與用藥習慣，以保障雙方權利義務

In [11]:
from peft import LoraConfig

# 設定 LoraConfig
lora_config = LoraConfig(
    inference_mode=False,  # 禁用推理模式（啟用訓練模式）
    lora_alpha=16,  # LoRA 的 alpha 超參數
    lora_dropout=0.05,  # LoRA dropout 超參數
    r=32,  # LoRA 等級
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "o_proj",
        "gate_proj",
        "down_proj",
        "up_proj"
    ],  # LoRA 應用的目標模組
    task_type="CAUSAL_LM"  # 任務類型
)


In [ ]:
# 添加 LoRA 配置到模型
lora_model = get_peft_model(base_model, lora_config)

lora_model.print_trainable_parameters()

trainable params: 48,627,712 || all params: 3,261,377,536 || trainable%: 1.4910


In [ ]:
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    labels = tokenizer(
        examples["target_text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )["input_ids"]
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]
    model_inputs["labels"] = labels
    return model_inputs

tr_tokenized_dataset = tr_processed_dataset.map(tokenize_function, batched=True)
vd_tokenized_dataset = vd_processed_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=lora_model,
    padding=True,
    max_length=512
)

# 訓練參數
training_args = TrainingArguments(
    output_dir="./lora_output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    learning_rate=1e-4,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    fp16=True,
    bf16=False,  
    report_to="none",
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tr_tokenized_dataset,
    eval_dataset=vd_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

Map: 100%|██████████| 1355/1355 [00:00<00:00, 7569.06 examples/s]
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_269012/4266855846.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2699: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.024100,0.002923
1000,0.023000,0.002902
1500,0.023100,0.002859
2000,0.023600,0.002861
2500,0.023000,0.002842
3000,0.022600,0.002834


/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67838336-4c6358f6042e6fde39e79a5e;1b06b3e2-4b3d-4875-b6ad-2ed334704fb3)

Cannot access gated repo for url https://huggingface.co/lianghsun/Llama-3.2-Taiwan-3B-Instruct/resolve/main/config.json.
Access to model lianghsun/Llama-3.2-Taiwan-3B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in lianghsun/Llama-3.2-Taiwan-3B-Instruct.
  warnings.warn(
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in lianghsun/Llama-3.2-Taiwan-3B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2699: UserWarni

TrainOutput(global_step=3048, training_loss=0.5842147354926337, metrics={'train_runtime': 3331.8869, 'train_samples_per_second': 7.32, 'train_steps_per_second': 0.915, 'total_flos': 2.1474428209279795e+17, 'train_loss': 0.5842147354926337, 'epoch': 1.9990979909799098})

In [16]:
lora_model.save_pretrained("./lora_finetuned_model_6")
tokenizer.save_pretrained("./lora_finetuned_model_6")

/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67838d30-5d4c798942fc490065d6232a;a01255e6-8e26-4b37-bdd4-b93eccb12e98)

Cannot access gated repo for url https://huggingface.co/lianghsun/Llama-3.2-Taiwan-3B-Instruct/resolve/main/config.json.
Access to model lianghsun/Llama-3.2-Taiwan-3B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in lianghsun/Llama-3.2-Taiwan-3B-Instruct.
  warnings.warn(
/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in lianghsun/Llama-3.2-Taiwan-3B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


('./lora_finetuned_model_6/tokenizer_config.json',
 './lora_finetuned_model_6/special_tokens_map.json',
 './lora_finetuned_model_6/tokenizer.json')

In [ ]:
from peft import PeftModel
from transformers import LlamaForCausalLM

model_name_or_path = "lianghsun/Llama-3.2-Taiwan-3B-Instruct"

lora_model = PeftModel.from_pretrained(base_model, "./lora_finetuned_model_6")

full_model = lora_model.merge_and_unload()

full_model.save_pretrained("./lora_finetuned_model_6")
tokenizer.save_pretrained("./lora_finetuned_model_6")


/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./lora_finetuned_model_6/tokenizer_config.json',
 './lora_finetuned_model_6/special_tokens_map.json',
 './lora_finetuned_model_6/tokenizer.json')

In [2]:
prompt = (
            '你是閱讀理解選擇題的答題者，請先理解以下文章的語意、時間、人物，'
            f'文章:\n 明年元旦起，交通部新規定，汽車輪胎胎紋深度將納入定期檢驗項目之一，一旦深度未達1.6公厘，及1個月內沒換胎，將會被吊銷牌照。民眾除了定期檢測胎紋，也可以自已用10元硬幣檢測，只要看的見國父像衣領下緣，表示該換輪胎了。\n'
            f'問題: 汽車胎紋未到達多少公厘將會被吊銷牌照?\n'
            f'選項：「選項1：1.5公厘\n選項2：1.4公厘\n選項3：1.3公厘\n選項4：1.6公厘」\n'
            "請問上面問題的正確答案是哪一個選項?\n"
            ) 
prompt = '你肚子餓嗎?'
result = qa_pipeline(prompt, max_new_tokens=100 , temperature=0.1, return_full_text=False)
result[0]['generated_text']

''

In [ ]:
from transformers import pipeline
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from safetensors.torch import save_file
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name_or_path = "lianghsun/Llama-3.2-Taiwan-3B-Instruct"

token = "yout_key"

finetuned_tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_auth_token=token)
model_path = "./lora_finetuned_model_6"
finetuned_model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto")

# pipeline
qa_pipeline = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
)

answer_list = []

try:
    with open("./AI1000.json", "r", encoding="utf-8") as f:
        data = json.load(f)
except Exception as e:
    print(f"Error reading JSON file: {e}")
    raise

ai1000_data = pd.DataFrame(data)

def generate_prompt(text):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{text}

### Response:"""

for _, row in ai1000_data.iterrows():

    text = (
        f"文章: 【{row['文章']}】\n"
        f"問題: {row['問題']}\n"
        f"選項如下：\n選項1：{row['選項1']}\n選項2：{row['選項2']}\n選項3：{row['選項3']}\n選項4：{row['選項4']}\n"
        "根據以上「文章」中提到內容的「問題」正確答案應該1~4中第幾個「選項」，請嚴格遵守僅回答1,2,3,4中的其中一個數字，不要多做說明?"
    )
    
    prompt = generate_prompt(text)
    input_ids = finetuned_tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    
    with torch.no_grad():
        output_ids = finetuned_model.generate(
            input_ids=input_ids,
            max_new_tokens=128,
            temperature=0.1,
            top_k=20,
            top_p=0.9,
            repetition_penalty=1.15
        ).cuda()

    output = finetuned_tokenizer.decode(output_ids[0], skip_special_tokens=True, return_full_text=False)
    result = output.split("### Response:")[-1].strip()

    print(f"題號: {row['題號']},\n 文章: {row['文章']}\n,問題: {row['問題']}\n 選項: {row['選項1']}、{row['選項2']}、{row['選項3']}、{row['選項4']}\n 模型回答: {result}")
    answer = {
        "題號": row['題號'],
        "問題": row['問題'],
        "選項": [row['選項1'], row['選項2'], row['選項3'], row['選項4']],
        "模型回答": result
    }
    answer_list.append(answer)

answer_output_path = "./llm_predict_result.json"

with open(answer_output_path, 'w', encoding='utf-8') as f:
    json.dump(answer_list, f, ensure_ascii=False, indent=4)

print(f"JSON 文件: {answer_output_path}")

/home/shuof/.conda/envs/llm/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


題號: 10023321,
 文章: 尹雪艷非常迷人，名氣大了，難免招忌。一些同行的姊妹看了眼紅，就到處放話：尹雪艷的八字帶著重煞，犯了白虎，沾上的人，輕者敗家，重者人亡。但是這一點反而增加了她的神秘感，讓她名氣更響亮。
,問題: 請問根據此段文章，下面哪一個選項與「尹雪艷八字犯重煞」無關？
 選項: 增加尹雪艷神秘感、同行因忌妒而放話、會讓親近者家敗人亡、忠孝仁愛信義和平
 模型回答: 根據文章內容，「安惡火爆烈性 八字犯重煞」並未被提及，因此選項4「忠孝仁愛信義和平」與「安惡火爆烈性 八字犯重煞」無關。

正確答案為：**4**。但需注意，問題中列出的選項並未包含「忠孝仁愛信義和平」這個選項，所以若要從給定選項中選出最接近之答項，最接近的是 **選項 4**，但實際上


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


題號: 10076124,
 文章: 男：請問，苗栗的縣立圖書館在哪裡？
女：就沿著門口那條路走到底，再向左轉就到了。
男：會很難找嗎？
女：不會。而且那裏有U-BIKE，你可以騎著繞一繞市區。可以到鐵道博物館走一走。
男：好的，謝謝你告訴我。
女：其實苗栗市區不大。
男：市區是不大，但房子都蠻有歷史感的，我很喜歡。
,問題: 這位先生最欣賞苗栗什麼？
 選項: 有歷史感的房子。、方便的U-BIKE。、圖書館的藏書。、鐵道博物館。
 模型回答: 選項1

根據文章內容，這位先生最欣賞的是「有歷史感的房子」。因此，正確答案是選項1。 lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai lai


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

題號: 45277639,
 文章: 女：明天的行程怎麼安排？ 男：白天先去西湖邊賞荷花，下午去雷峰塔，晚上去逛清河坊夜市。但如果下雨的話，在雷峰塔看完夕陽就得回飯店了。 女：那希望明天一整天都是好天氣。
,問題: 請問從以上對話，可以得知什麼訊息？
 選項: 明天一定會下雨、無論天氣好不好，他們都要去雷峰塔、今天的天氣很好、就算天氣很好，他們也不去逛夜市
 模型回答: 選項1：明天一定會下雨

根據文章中的描述，明天的天氣預報並未明確提到，但男方建議了幾個活動方案，而這些方案都是基於天氣良好的前提。因此，雖然不能確定明天是否一定會下雨，但可以推測出，如果明天的天氣不佳，則男方可能會依照備用方案行動。然而，根據問題要求，我們需選擇最合理的答案，因此選項1是最接近真實情況的選項


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


題號: 45377503,
 文章: 佛洛伊德在《夢的解析》，提出所謂的「伊底帕斯情節」，他認為那是一種願望的達成──我們童年時期的願望的達成。他認為，大多數的人並不會變成心理疾病，而能成功地將母親的性衝動逐次收回，並且漸漸忘掉對父親的嫉妒心。我們就是這樣子，並且盡其所能地給予潛在地抑制。
,問題: 何者為這段話的主旨？
 選項: 殺父娶母無法解釋。、遇到家庭暴力不要聲張。、原始願望可以抑制。、原始願望無法抑制。
 模型回答: 選項1

這段話的主旨是關於弗洛伊德提出的「伊底帕斯情節」，以及人類如何透過願望來處理早期經驗中的矛盾和衝突。弗洛伊德認為，人類可以透過願望來達成童年時期的目標，即使這些目標可能與現實環境相矛盾。因此，這段話並不能直接說明「殺父娶母」或「遇到家庭暴力不要聲張」的原因，而是強調了願


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


題號: 45424161,
 文章: 建築名家王大閎赴美學習建築時，對簡約主義印象深刻。往後，在他的建築作品中處處透露著中國傳統建築美學與簡約主義的品味。如位於台北市建國南路的自家住宅，便是中西結合的最佳代表。無獨有偶，貝聿銘主持設計的東海大學路思義教堂，也是素樸美學具體呈現的例子。
,問題: 請問根據文章，下列何者正確？

 選項: 王大閎赴美學習平面廣告設計
、貝聿銘和王大閎的建築觀大相逕庭
、王大閎的代表作是東海大學路思義教堂
、王大閎和貝聿銘的建築觀皆含有質樸之美
 模型回答: 選項2：貝聿銘和王大閎的建築觀大相逕庭

根據文章內容，王大閎在赴美學習平面廣告設計後，對簡約主義留下了深刻的印象。然而，文章並未提到王大閎和貝聿銘的建築觀大相逕庭，因此選項2不正確。

選項1：王大閎赴美學習平面廣告設計

王大閎確實曾赴美學習平面廣告


In [7]:
import json
import csv
import re

# 加载 JSON 文件
with open('./llm_predict_result.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

with open('./AI1000.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

# 将 question 转为字典，方便通过题号查找
question_dict = {q["題號"]: q for q in questions}

# 处理数据
rows = []
for entry in data:
    id_ = entry["題號"]
    model_answer = str(entry["模型回答"])  # 确保为字符串

    # 优先检查是否包含数字 1-4
    answer_match = re.search(r'[1-4]', model_answer)
    if answer_match:
        answer = answer_match.group()
    else:
        # 如果没有数字，检查选项文字匹配
        if id_ in question_dict:
            options = [
                str(question_dict[id_].get("選項1", "")),  # 确保为字符串
                str(question_dict[id_].get("選項2", "")),
                str(question_dict[id_].get("選項3", "")),
                str(question_dict[id_].get("選項4", ""))
            ]
            found = False
            for idx, option in enumerate(options, start=1):
                if option in model_answer:
                    answer = str(idx)
                    found = True
                    break
            if not found:
                answer = "無法判斷"
        else:
            answer = "無效題號"

    rows.append({"ID": id_, "Answer": answer})

# 保存为 CSV 文件
output_path = "./0111_3_processed_answers.csv"
with open(output_path, 'w', encoding='utf-8', newline='') as csvfile:
    fieldnames = ["ID", "Answer"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print(f"结果已保存到 {output_path}")


结果已保存到 ./0111_3_processed_answers.csv
